In [79]:
Min_Int = -2**31
Max_Int = 2**31-1

import ast

from enum import Enum
class Translate_Actions (Enum):
    PROC_DEC      = 1
    FUNC_DEC      = 2
    FUNC_RESULT   = 3
    IN_PARAM      = 4
    OUT_PARAM     = 5
    IN_OUT_PARAM  = 6
    IN_GLOBAL     = 7
    IN_OUT_GLOBAL = 8
    LOCAL_RW      = 9
    LOCAL_RO      = 10
    EVAL          = 11
    ASSIGN        = 12
    DEFINE_CONST  = 13
    DEFER_CONST   = 14
    IF_COND       = 15
    @classmethod
    def pos (cls, act):
        return act.value - 1
        
Action = Enum('Action', ['PROC_DEC', 'FUNC_DEC', 'FUNC_RESULT', 
                         'IN_PARAM', 'OUT_PARAM', 'IN_OUT_PARAM',
                         'IN_GLOBAL', 'OUT_GLOBAL', 'IN_OUT_GLOBAL', 
                         'LOCAL_RW', 'LOCAL_RO',
                         'EVAL', 'ASSIGN', 'DEFINE_CONST', 'DEFER_CONST', 'IF_COND'])

Mode = Enum('Mode', ['IN_PARAM', 'OUT_PARAM', 'IN_OUT_PARAM', 'IN_GLOBAL', 'IN_OUT_GLOBAL', 'LOCAL_RW', 'LOCAL_RO'])

vars        = {}
var_count   = 0
state_stack = []
frame_ptr   = 0
frame       = 0

branch_stack = []
branch_ptr   = 0

class Var_State:
    def __init__(self, name, mode, well_formed, scalar, lb, ub, invariant):
        self.name        = name
        self.mode        = mode
        self.well_formed = well_formed
        self.scalar      = scalar
        self.lb          = lb
        self.ub          = ub
        self.invariant   = invariant
        self.evaluated   = False
        self.assigned    = False
        self.updated     = False
        
    def print_var_state(self):
        print('name =', self.name)
        print('mode =', self.mode)
        print('well_formed =', self.well_formed)
        print('scalar =', self.scalar)
        print('lb =', self.lb)
        print('ub =', self.ub)
        print('invariant =', self.invariant)
        print('evaluated =', self.evaluated)
        print('assigned =', self.assigned)
        print('updated =', self.updated)
            
def print_state_stack(stack):
    for i in range(len(stack)):
        stack[i].print_var_state()
        print('----------------------')

def new_frame():
    result = []
    for i in range (frame_ptr-frame, frame):
        result = result + [state_stack[i]]
        result[-1].updated = False
    return result



In [80]:
print_state_stack(state_stack)

In [81]:
# Each statement is a list of actions on variables
def process_statement (actions):
    global vars, var_count, state_stack, frame_ptr, frame, branch_stack, branch_ptr
    for act in actions:
        if act[0] == Action.PROC_DEC:
            frame_ptr = frame_ptr + frame
        elif act[0] == Action.IN_PARAM:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.IN_PARAM, 
                                                   well_formed=(True, []),
                                                   scalar=True, 
                                                   lb=Min_Int, ub=Max_Int,
                                                   invariant=True)]
            frame = frame + 1
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.OUT_PARAM:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.OUT_PARAM, 
                                                   well_formed=(False, []), 
                                                   scalar=True,
                                                   lb=None, ub=None,
                                                   invariant=False)]
            frame = frame + 1
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.IN_OUT_PARAM:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.IN_OUT_PARAM, 
                                                   well_formed=(True, []), 
                                                   scalar=True, 
                                                   lb=Min_Int, ub=Max_Int,
                                                   invariant=False)]
            frame = frame + 1        
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.IN_GLOBAL:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.IN_GLOBAL, 
                                                   well_formed=(True, []),
                                                   scalar=True, 
                                                   lb=Min_Int, ub=Max_Int,
                                                   invariant=True)]
        elif act[0] == Action.OUT_GLOBAL:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.OUT_GLOBAL, 
                                                   well_formed=(False, []), 
                                                   scalar=True, 
                                                   lb=None, ub=None,
                                                   invariant=False)]
            frame = frame + 1
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.IN_OUT_GLOBAL:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.IN_OUT_GLOBAL, 
                                                   well_formed=(True, []),
                                                   scalar=True, 
                                                   lb=Min_Int, ub=Max_Int,
                                                   invariant=False)]
            frame = frame + 1        
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.LOCAL_RW:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.LOCAL_RW, 
                                                   well_formed=(False, []), 
                                                   scalar=True, 
                                                   lb=None, ub=None,
                                                   invariant=False)]
            frame = frame + 1
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.LOCAL_RO:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.LOCAL_RO, 
                                                   well_formed=(False, []), 
                                                   scalar=True, 
                                                   lb=None, ub=None,
                                                   invariant=False)]
            frame = frame + 1
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.EVAL:
            state_stack[vars[act[1]]].evaluated=True
        elif act[0] == Action.ASSIGN:
            state_index = vars[act[1]]
            if state_stack[state_index].assigned:
                state_stack[state_index].updated = True
            else:
                state_stack[state_index].assigned = True

            if state_stack[state_index].well_formed[0] == False:
                state_stack[state_index].well_formed = (True, state_stack[state_index].well_formed[1] + [branch_stack[-1][2]])
        elif act[0] == Action.IF_COND:
            branch_stack = branch_stack + [(Action.IF_COND, False, ast.parse(act[1]), frame, frame_ptr)]
            branch_ptr = branch_ptr + 1
            state_stack = state_stack + new_frame()
            frame_ptr = frame_ptr + frame
            branch_stack = branch_stack + [(Action.IF_COND, True, ast.parse(act[1]), frame, frame_ptr)]
            branch_ptr = branch_ptr + 1
            # May be able to adjust bounds of variables used in condition.
            # for instance, if condition is 'x > y + 10' then x lower bound is lb(y) + 10
        else:
            print("Unsupported") 
            
        

In [82]:
process_statement([(Action.PROC_DEC, 'p'), 
                   (Action.IN_OUT_PARAM, 'x'),
                  (Action.IN_OUT_PARAM, 'y')])
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = False
assigned = False
updated = False
----------------------


In [83]:
vars

{'x': 0, 'y': 1}

In [84]:
process_statement([(Action.LOCAL_RW, 'temp')])
print_state_stack(state_stack)               

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
lb = None
ub = None
invariant = False
evaluated = False
assigned = False
updated = False
----------------------


In [85]:
process_statement([(Action.EVAL, 'x'),
                  (Action.EVAL, 'y'),
                  (Action.IF_COND, 'x > y + 10')])
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
lb = None
ub = None
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOC

In [86]:
vars

{'x': 0, 'y': 1, 'temp': 2}

In [87]:
process_statement([(Action.EVAL,'x'), (Action.ASSIGN, 'temp')])

In [88]:
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (True, [<ast.Module object at 0x114439b90>])
scalar = True
lb = None
ub = None
invariant = False
evaluated = False
assigned = True
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
------------------

In [89]:
branch_stack

[(<Action.IF_COND: 16>, False, <ast.Module at 0x11384e790>, 3, 0),
 (<Action.IF_COND: 16>, True, <ast.Module at 0x114439b90>, 3, 3)]

In [61]:
frame_ptr

3

In [21]:
from collections import namedtuple

Var_State = namedtuple('Var_State', ['id', 'well_formed', 'evaluated', 'assigned', 'invariant', 'updated',
                                     'scalar', 'lb', 'ub'
                                    ])

Min_Int = -2**31
Max_Int = 2**31-1

# stack frame
frame = 3
x = 0
y = 1
temp = 2

frame_ptr = 0

block_condition = ['True']


In [8]:
# Line 1: procedure P (X : in out Integer; Y : in out Integer) is
# Translation
[(Action.IN_OUT_PARAM, x),
 (Action.IN_OUT_PARAM, y))]
# IN_OUT_PARAM X
# IN_OUT_PARAM Y
stack = [Var_State('x', well_formed=True,evaluated=False,assigned=False,invariant=True,updated=False,
                  scalar=True, lb=Min_Int, ub=Max_Int),
         Var_State('y', well_formed=True,evaluated=False,assigned=False,invariant=True,updated=False,
                   scalar=True, lb=Min_Int, ub=Max_Int),
         Var_State('temp', None, None, None, None, None, None, None, None)]
stack

[Var_State(id='x', well_formed=True, evaluated=False, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='y', well_formed=True, evaluated=False, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='temp', well_formed=None, evaluated=None, assigned=None, invariant=None, updated=None, scalar=None, lb=None, ub=None)]

In [9]:
# Line 2:     Temp: Integer;  
# Translation
# VAR_DEC Temp
stack[temp+frame_ptr] = Var_State('temp',well_formed=False,evaluated=False,assigned=False,invariant=False,
                                 updated=False, scalar=True, lb=None, ub=None)
stack

[Var_State(id='x', well_formed=True, evaluated=False, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='y', well_formed=True, evaluated=False, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='temp', well_formed=False, evaluated=False, assigned=False, invariant=False, updated=False, scalar=True, lb=None, ub=None)]

In [10]:
# Line 3:     if X > Y + 10 then  
# EVAL x
stack[x+frame_ptr] = Var_State('x', well_formed=True, evaluated=True, assigned=False, invariant=True,
                              updated=False, scalar=True, lb=Min_Int, ub=Max_Int)
# EVAL y
stack[y+frame_ptr] = Var_State('y', well_formed=True, evaluated=True, assigned=False, invariant=True,
                              updated=False, scalar=True, lb=Min_Int, ub=Max_Int)
stack


[Var_State(id='x', well_formed=True, evaluated=True, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='y', well_formed=True, evaluated=True, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='temp', well_formed=False, evaluated=False, assigned=False, invariant=False, updated=False, scalar=True, lb=None, ub=None)]

In [11]:
# Line 3.1:    if X > Y + 10 then  
# IF_CONDITION_FALSE
block_condition = block_condition + [block_condition[-1] + ' and ' 'not x > y + 10']

In [12]:
# Line 3.2:     if X > Y + 10 then  
# IF_CONDITION_TRUE
block_condition = block_condition + [block_condition[-2] + ' and ' 'x > y + 10']

In [13]:
block_condition

['True', 'True and not x > y + 10', 'True and x > y + 10']

In [14]:
# Line 3.3:     if X > Y + 10 then
# THEN_BRANCH
stack = stack + stack[frame_ptr:]
frame_ptr = frame_ptr + frame 
stack

[Var_State(id='x', well_formed=True, evaluated=True, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='y', well_formed=True, evaluated=True, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='temp', well_formed=False, evaluated=False, assigned=False, invariant=False, updated=False, scalar=True, lb=None, ub=None),
 Var_State(id='x', well_formed=True, evaluated=True, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='y', well_formed=True, evaluated=True, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='temp', well_formed=False, evaluated=False, assigned=False, invariant=False, updated=False, scalar=True, lb=None, ub=None)]

In [15]:
frame_ptr

3

In [18]:
# Line 4:         Temp := X;  
# Translation
# EVAL X
# ASSIGN Temp
stack [temp+frame_ptr] = Var_State(id='temp', well_formed=True, evaluated=False, assigned=False, 
                                   invariant=False, updated=False, scalar=True, lb=Min_Int, ub=Max_Int)
stack

[Var_State(id='x', well_formed=True, evaluated=True, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='y', well_formed=True, evaluated=True, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='temp', well_formed=False, evaluated=False, assigned=False, invariant=False, updated=False, scalar=True, lb=None, ub=None),
 Var_State(id='x', well_formed=True, evaluated=True, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='y', well_formed=True, evaluated=True, assigned=False, invariant=True, updated=False, scalar=True, lb=-2147483648, ub=2147483647),
 Var_State(id='temp', well_formed=True, evaluated=False, assigned=False, invariant=False, updated=False, scalar=True, lb=-2147483648, ub=2147483647)]

In [129]:
# Line 5:         X := Y  
# EVAL Y
# ASSIGN X
stack[x+frame_ptr] = Var_State(id='x', well_formed=True, evaluated=True, assigned=True, invariant=False)
stack

[Var_State(id='x', well_formed=True, evaluated=True, assigned=False, invariant=True),
 Var_State(id='y', well_formed=True, evaluated=True, assigned=False, invariant=True),
 Var_State(id='temp', well_formed=False, evaluated=False, assigned=False, invariant=False),
 Var_State(id='x', well_formed=True, evaluated=True, assigned=True, invariant=False),
 Var_State(id='y', well_formed=True, evaluated=True, assigned=False, invariant=True),
 Var_State(id='temp', well_formed=True, evaluated=False, assigned=False, invariant=False)]

In [ ]:
# Line 6:        Y := X;  
# READ X
# ASSIGN Y

x4 = (defined, read, unwritten, invariant, ('x_in', 'x_in'), 'x > y + 10')
y4 = (defined, read, unwritten, invariant, ('y_in', 'y_in'), 'x > y + 10')
temp4 = (defined, unread, written, variant, (x_in, x_in), 'x > y + 1')
state3_2 = [x3_2, y3_2, temp3_2]
stack3_2 = [state2, state3, state3_1, state3_2]